In [90]:
import boto3
import pandas as pd
from sagemaker import get_execution_role

from sqlalchemy import create_engine
import psycopg2 
import psycopg2.extras
import io

In [109]:
def list_data(folder):
    """
    List all available data sets in the folder
    """
    s3 = boto3.resource('s3')
    bucket='cp-hackathon-data'
    hackathon_data = s3.Bucket(bucket)
    keys = []
    for obj in hackathon_data.objects.filter(Prefix=folder):
        keys.append(obj.key)
    return keys
def get_data(key):
    role = get_execution_role()
    bucket='cp-hackathon-data'
    data_location = 's3://{}/{}'.format(bucket, key)
    return pd.read_csv(data_location)
def add_to_db(df, conn,table='import.schools'):
    df_columns = list(df)
    # create (col1,col2,...)
    columns = ",".join(df_columns)

    # create VALUES('%s', '%s",...) one '%s' per column
    values = "VALUES({})".format(",".join(["%s" for _ in df_columns])) 

    #create INSERT INTO table (columns) VALUES('%s',...)
    insert_stmt = "INSERT INTO {} ({}) {}".format(table,columns,values)

    cur = conn.cursor()
    psycopg2.extras.execute_batch(cur, insert_stmt, df.values)
    conn.commit()
    cur.close()

In [43]:
ovitrap = get_data('RedCross/ovitrap_data.csv')
ovitrap_province = get_data('RedCross/ovitrap_data_monthly_province.csv')
weather_province = get_data('RedCross/weather_data_monthly_province.csv')

Couldn't call 'get_role' to get Role ARN from role name cp-sagemaker-hackathon-role to get Role path.
Couldn't call 'get_role' to get Role ARN from role name cp-sagemaker-hackathon-role to get Role path.
Couldn't call 'get_role' to get Role ARN from role name cp-sagemaker-hackathon-role to get Role path.


In [108]:
weather_province

,adm_level,date,JAXA_GPM_L3_GSMaP_v6_operational_hourlyPrecipRateGC,MODIS_006_MOD11A1_LST_Day_1km,MODIS_006_MOD11A1_LST_Night_1km,MODIS_006_MYD13A1_EVI,NASA_FLDAS_NOAH01_C_GL_M_V001_Qair_f_tavg,NASA_FLDAS_NOAH01_C_GL_M_V001_Rainf_f_tavg,NASA_FLDAS_NOAH01_C_GL_M_V001_SoilMoi00_10cm_tavg,NASA_FLDAS_NOAH01_C_GL_M_V001_SoilTemp00_10cm_tavg,NASA_FLDAS_NOAH01_C_GL_M_V001_Tair_f_tavg,NASA_FLDAS_NOAH01_C_GL_M_V001_Wind_f_tavg
0,Abra,2012-06-01,0.430867,15043.5540,14639.8580,4180.8150,0.016763,0.000172,0.425508,296.31796,297.09457,4.977150
1,Abra,2012-07-01,0.373154,15020.1080,14632.6040,3662.6284,0.016362,0.000186,0.428822,295.84708,296.72488,3.478785
2,Abra,2012-08-01,0.620665,14917.0320,14486.0460,3183.5750,0.016541,0.000245,0.432180,295.66570,296.15265,6.328744
3,Abra,2012-09-01,0.106620,14980.0205,14696.8920,4785.5340,0.015967,0.000099,0.419719,296.26144,296.81122,3.009687
4,Abra,2012-10-01,0.053113,14986.9280,14666.7160,4770.0083,0.014127,0.000039,0.390687,295.36737,295.53925,4.924677
5,Abra,2012-11-01,0.017088,15037.7130,14696.7870,4477.5903,0.014546,0.000031,0.364914,296.01535,296.57947,3.134883
6,Abra,2012-12-01,0.024846,15016.3850,14658.5100,4064.4753,0.013407,0.000033,0.339067,295.31958,295.44562,4.674698
7,Abra,2013-01-01,0.011827,14978.5050,14557.2510,3921.3790,0.011683,0.000008,0.288960,294.48914,293.92303,5.225402
8,Abra,2013-02-01,0.007670,15112.7705,14673.9380,3848.7920,0.012277,0.000005,0.234911,296.74030,295.98770,4.540266
9,Abra,2013-03-01,0.006974,15173.5000,14697.5970,3844.4028,0.012938,0.000008,0.219408,298.04120,296.82443,3.457057


In [44]:
ovitrap['point_id'] = (ovitrap.groupby(by =['latitude','longitude']).cumcount()==0).astype(int)
ovitrap['point_id'] = ovitrap['point_id'].cumsum()

In [110]:
conn = ps.connect(dbname="mosquito-spatial", user="health_hackathon", password="Health_100%",host='mosquitopie.cluster-crqnajfhzbab.eu-west-1.rds.amazonaws.com')
add_to_db(ovitrap[cols],conn,'analysis.schools_pointid')


,Unnamed: 0,date,id,latitude,longitude,name,value,point_id
0,0,2012-06-19,3,14.504400,121.056000,Kapitan Jose Cardones Memorial School,17.31,1
1,1,2012-07-03,3,14.504400,121.056000,Kapitan Jose Cardones Memorial School,15.52,1
2,2,2012-07-17,3,14.504400,121.056000,Kapitan Jose Cardones Memorial School,30.00,1
3,3,2012-07-24,3,14.504400,121.056000,Kapitan Jose Cardones Memorial School,22.64,1
4,4,2012-09-11,3,14.504400,121.056000,Kapitan Jose Cardones Memorial School,28.13,1
5,5,2012-09-18,3,14.504400,121.056000,Kapitan Jose Cardones Memorial School,25.76,1
6,6,2012-09-25,3,14.504400,121.056000,Kapitan Jose Cardones Memorial School,15.71,1
7,7,2012-10-02,3,14.504400,121.056000,Kapitan Jose Cardones Memorial School,18.75,1
8,8,2012-10-09,3,14.504400,121.056000,Kapitan Jose Cardones Memorial School,21.54,1
9,9,2012-10-16,3,14.504400,121.056000,Kapitan Jose Cardones Memorial School,26.32,1


In [116]:
conn = ps.connect(dbname="mosquito-spatial", user="health_hackathon", password="Health_100%",host='mosquitopie.cluster-crqnajfhzbab.eu-west-1.rds.amazonaws.com')
query = """
SELECT *
FROM analysis.schools_adm_boundaries
"""
df_mapping = pd.read_sql(query, conn)
df_mapping.point_id = df_mapping.point_id.astype(int)
df_mapping

,point_id,latitude,longitude,adm2_en
0,7546,17.819566,120.802585,Abra
1,7606,17.619125,120.685780,Abra
2,7658,17.563315,120.989276,Abra
3,7669,17.733497,120.708136,Abra
4,7673,17.646691,120.710841,Abra
5,7677,17.465584,120.600280,Abra
6,7678,17.540384,120.519153,Abra
7,7682,17.555695,120.563836,Abra
8,7808,17.440104,120.802226,Abra
9,7828,17.453067,120.756794,Abra


In [123]:
tmp = ovitrap.merge(df_mapping,on='point_id',how='left')
len(tmp.loc[tmp.adm2_en.isnull()].point_id.unique())


78

In [104]:
point_table = ovitrap[['point_id','latitude','longitude']].drop_duplicates(subset='point_id')


In [107]:
conn = ps.connect(dbname="mosquito-spatial", user="health_hackathon", password="Health_100%",host='mosquitopie.cluster-crqnajfhzbab.eu-west-1.rds.amazonaws.com')
add_to_db(point_table,conn)